## Generate 1-way prompt enhanced audio for the HouseX dataset

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/housex-audio-embeds-experiments/housex-audio-benchmarking/gpt_saved_outputs.pkl
/kaggle/input/housex-audio-embeds-experiments/housex-audio-benchmarking/clap_fused_housex_saved_embeds_dict.pkl


## Construct genre-wise prompts from the corpus

In [2]:
import pickle

with open("/kaggle/input/housex-audio-embeds-experiments/housex-audio-benchmarking/gpt_saved_outputs.pkl", "rb") as f:
    corpus_dict = pickle.load(f)

In [3]:
for k in corpus_dict:
    l = corpus_dict[k].split(",")
    l = [x.strip() for x in l]
    corpus_dict[k] = l

### Prompt format: `A <GENRE> song that <DESCRIPTOR>`

In [4]:
prompt_dict = {}

for k in corpus_dict:
    if k not in prompt_dict:
        prompt_dict[k] = []
    for desc in corpus_dict[k]:
        annotated_prompt = f"A {k.replace('_', ' ')} song that {desc}."
        prompt_dict[k].append(annotated_prompt)

In [5]:
prompt_dict

{'melodic_house': ['A melodic house song that energizes.',
  'A melodic house song that transitions.',
  'A melodic house song that blends.',
  'A melodic house song that admires.',
  'A melodic house song that recommends.',
  'A melodic house song that impresses.',
  'A melodic house song that utilizes.',
  'A melodic house song that builds.',
  'A melodic house song that surprises.',
  'A melodic house song that stuns.'],
 'future_house': ['A future house song that unveils.',
  'A future house song that moulds.',
  'A future house song that brims.',
  'A future house song that introduces.',
  'A future house song that revamps.',
  'A future house song that executes.',
  'A future house song that incorporates.',
  'A future house song that lacks.',
  'A future house song that fuses.',
  'A future house song that imagines.',
  'A future house song that collaborates.',
  'A future house song that surprises.',
  'A future house song that breaks.',
  'A future house song that smashes.',
 

## Load model and define the methods to generate audio and embeddings 

In [6]:
!pip install -q --upgrade diffusers transformers accelerate

In [7]:
from diffusers import AudioLDM2Pipeline
import torch

torch.manual_seed(42)

repo_id = "cvssp/audioldm2-music"
pipe = AudioLDM2Pipeline.from_pretrained(repo_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/222M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/221M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/11 [00:00<?, ?it/s]

2024-02-07 05:25:29.541273: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 05:25:29.541425: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 05:25:29.686897: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
generator = torch.Generator("cuda").manual_seed(42)

In [9]:
import scipy
from scipy.io import wavfile
import librosa

def load_audio_file(file_path):
    sr, origin_data = wavfile.read(file_path)
    origin_type = origin_data.dtype
    resampled_data = librosa.resample(origin_data.T.astype('float'), orig_sr = sr, target_sr = 48000) # transpose array to librosa shape and sampling rate of 48000
    resampled_data = librosa.to_mono(resampled_data)        
    resampled_data = resampled_data.T.astype(origin_type) # transpose back to scipy.io.wavfile shape
    data_np = np.array(resampled_data)
    return data_np

def generate_and_save_audio(prompt, output_file_path, negative_prompt = "low quality, monotonous, boring"):
    global pipe
    audio = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=500,
        audio_length_in_s=10.0,
    ).audios[0]
    
    scipy.io.wavfile.write(output_file_path, rate=16000, data=audio)

In [10]:
from datasets import load_dataset
from transformers import pipeline
from datasets import load_dataset
from transformers import ClapModel, ClapProcessor

In [11]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [12]:
model = ClapModel.from_pretrained("laion/clap-htsat-fused").to(device)
processor = ClapProcessor.from_pretrained("laion/clap-htsat-fused")

config.json:   0%|          | 0.00/5.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


preprocessor_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

## Generate embeddings for all prompts

In [13]:
from tqdm import tqdm

embeddings_dict = {}


for k in prompt_dict:
    if k not in embeddings_dict:
        embeddings_dict[k] = []
    audio_index = 0

    for prompt in tqdm(prompt_dict[k]):
        output_file_path = f"{k}_generated_{audio_index}.wav"
        generate_and_save_audio(prompt, output_file_path)
        audio_inputs_np = load_audio_file(output_file_path)
        clap_inputs = processor(audios=audio_inputs_np, sampling_rate=48000, return_tensors="pt").to(device)
        clap_outputs = model.get_audio_features(**clap_inputs)
        audio_embeds = clap_outputs[0].detach().cpu()
        embeddings_dict[k].append(audio_embeds)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

 10%|█         | 1/10 [01:08<10:16, 68.55s/it]

  0%|          | 0/500 [00:00<?, ?it/s]

 20%|██        | 2/10 [02:06<08:16, 62.04s/it]

...

 90%|█████████ | 18/20 [17:22<01:56, 58.05s/it]

  0%|          | 0/500 [00:00<?, ?it/s]

 95%|█████████▌| 19/20 [18:20<00:58, 58.06s/it]

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 20/20 [19:18<00:00, 57.95s/it]


In [14]:
for k in embeddings_dict:
    print(k, len(embeddings_dict[k]))

melodic_house 10
future_house 20
progressive_house 20
bass_house 20


In [15]:
import pickle

with open("embeddings_audioldm_500_steps.pkl", "wb") as f:
    pickle.dump(embeddings_dict, f)